[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# UnSupervised Learning Methods

## Dimensionality Reduction - IsoMap

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 28/05/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethods/2023_01/0041DimensionalityReductionIsoMap.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import make_s_curve
from sklearn.manifold import Isomap, MDS

# Miscellaneous
import os
import math
from platform import python_version
import random
import urllib.request

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

# Searching face_data.mat github
DATA_FILE_URL = r'https://github.com/Mashimo/datascience/raw/master/datasets/face_data.mat'
DATA_FILE_URL = r'https://github.com/SpencerKoevering/DRCapstone/raw/main/Isomap_face_data.mat'
DATA_FILE_URL = r'https://github.com/jasonfilippou/DimReduce/raw/master/ISOMAP/face_data.mat'

DATA_FILE_NAME = r'IsoMapFaceData.mat'


In [ ]:
# Fixel Algorithms Packages


## Dimensionality Reduction by IsoMap

The IsoMap is a special case of the MDS approach where we try to approximate the geodesic distance by the shortest path distance.  
The geodesic distance is the distance on the low dimensional surface (Manifold) the data is assumed to lie on.  
Hence, by knowing it we can use the data native metric.

In this notebook:

 - We'll use the IsoMap algorithm to reduce the dimensionality of the data set.
 - We'll compare results of the IsoMap with the MDS algorithm with euclidean distance metric.  

In [ ]:
# Parameters

# Data
numRows  = 4
numCols  = 4
tImgSize = (64, 64)

# Model
numNeighbors    = 6
lowDim          = 2
metricType      = 'l2'

# Visualization
imgShift        = 5
numImgScatter   = 70


In [ ]:
# Auxiliary Functions

def PlotMnistImages(mX: np.ndarray, vY: np.ndarray = None, numRows: int = 1, numCols: int = 1, imgSize = (28, 28), randomChoice = True, hF = None):

    numSamples  = mX.shape[0]

    numImg = numRows * numCols

    # tFigSize = (numRows * 3, numCols * 3)
    tFigSize = (numCols * 3, numRows * 3)

    if hF is None:
        hF, hA = plt.subplots(numRows, numCols, figsize = tFigSize)
    else:
        hA = hF.axis
    
    hA = np.atleast_1d(hA) #<! To support numImg = 1
    hA = hA.flat

    
    for kk in range(numImg):
        if randomChoice:
            idx = np.random.choice(numSamples)
        else:
            idx = kk
        mI  = np.reshape(mX[idx, :], imgSize)
    
        hA[kk].imshow(mI, cmap = 'gray')
        hA[kk].tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, labelleft = False, labeltop = False, labelright = False, labelbottom = False)
        labelStr = f', Label = {vY[idx]}' if vY is not None else ''
        hA[kk].set_title(f'Index = {idx}' + labelStr)
    
    plt.show()

def PlotScatterData(mX: np.ndarray, vL: np.ndarray, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, markerSize: int = MARKER_SIZE_DEF, lineWidth: int = LINE_WIDTH_DEF, axisTitle: str = None):

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()
    
    vU = np.unique(vL)
    numClusters = len(vU)

    for ii in range(numClusters):
        vIdx = vL == vU[ii]
        hA.scatter(mX[vIdx, 0], mX[vIdx, 1], s = ELM_SIZE_DEF, edgecolor = EDGE_COLOR, label = ii)
    
    hA.set_xlabel('${{x}}_{{1}}$')
    hA.set_ylabel('${{x}}_{{2}}$')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.grid()
    hA.legend()

    return hA

def PlotScatterData3D(mX: np.ndarray, vL: np.ndarray = None, vC: np.ndarray = None, axesProjection: str = '3d', hA: plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, markerSize: int = MARKER_SIZE_DEF, lineWidth: int = LINE_WIDTH_DEF, axisTitle: str = None):

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize, subplot_kw = {'projection': axesProjection})
    else:
        hF = hA.get_figure()
    
    if vL is not None:
        vU = np.unique(vL)
        numClusters = len(vU)
    else:
        vL = np.zeros(mX.shape[0])
        vU = np.zeros(1)
        numClusters = 1

    for ii in range(numClusters):
        vIdx = vL == vU[ii]
        if axesProjection == '3d':
            hA.scatter(mX[vIdx, 0], mX[vIdx, 1], mX[vIdx, 2], s = ELM_SIZE_DEF, c = vC, alpha = 1, edgecolor = EDGE_COLOR, label = ii)
        else:
            hA.scatter(mX[vIdx, 0], mX[vIdx, 1], s = ELM_SIZE_DEF, c = vC, alpha = 1, edgecolor = EDGE_COLOR, label = ii)
    
    hA.set_xlabel('${{x}}_{{1}}$')
    hA.set_ylabel('${{x}}_{{2}}$')
    if axesProjection == '3d':
        hA.set_zlabel('${{x}}_{{3}}$')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.grid()
    hA.legend()

    return hA


## Generate / Load Data

In this notebook we'll use [IsoMap Face Data Set](https://web.archive.org/web/20160913051505/http://isomap.stanford.edu/datasets.html).    
This data set is composed with 698 images of size `64 x 64` of the same face.  
Each image is taken from a different angle: Vertical and Horizontal.

![](https://i.imgur.com/cNz811Y.png)

We'll download the data from GitHub (There are 3 URL above, one should work :-)).

* <font color='red'>(**?**)</font> What are the degrees of freedom of the underlying manifold of the data?
* <font color='red'>(**?**)</font> What's its shape?

In [ ]:
# Download Data
# This section downloads data from the given URL if needed.

if not os.path.exists(DATA_FILE_NAME):
    urllib.request.urlretrieve(DATA_FILE_URL, DATA_FILE_NAME)

In [ ]:
# Loading / Generating Data

# Dictionary of the data
# 'images' - The images.
# 'poses' - The angles.
dFaceData = sp.io.loadmat(DATA_FILE_NAME)
mX        = dFaceData['images'].T #<! Loading from MATLAB

numSamples, dataDim = mX.shape

print(f'The features data shape: {mX.shape}')
print(f'The features data type: {mX.dtype}')

* <font color='red'>(**?**)</font> Do we need to scale the data?
* <font color='blue'>(**!**)</font> Check the dynamic range of the data (Images).

In [ ]:
# Transpose each image (MATLAB -> Python)

for vX in mX:
    vX[:] = np.reshape(np.reshape(vX, tImgSize), (-1, ), order = 'F')

### Plot the Data

In [ ]:
# Plot the Data

PlotMnistImages(mX, numRows = numRows, numCols = numCols, imgSize = tImgSize)

## Applying Dimensionality Reduction - IsoMap

We'll use the IsoMap algorithm to approximate the data native manifold.  

One of the earliest (In ~2000 by Joshua B. Tenenbaum) approaches to manifold learning is the IsoMap algorithm, short for _Isometric Mapping_.  
IsoMap can be viewed as an extension of Multi Dimensional Scaling (MDS) or Kernel PCA.  
IsoMap seeks a lower-dimensional embedding which maintains geodesic distances between all points. 

![Isomap](https://github.com/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethod/19_DimensionalityReduction/Isomap.png?raw=true)

We'll use SciKit Learn's [`Isomap`](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.Isomap.html) class.

* <font color='brown'>(**#**)</font> The method is based on MDS which means there is no unique solution.
* <font color='brown'>(**#**)</font> The complexity of the algorithm is rather high hence there are many approximated steps.
* <font color='brown'>(**#**)</font> Behind the scene the SciKit Learn implementation approximate the geodesic distance using a Kernel (So the solution is equivalent to K-PCA).



* <font color='red'>(**?**)</font> What do we send in for production from this model?

In [ ]:
# Apply the IsoMap

# Construct the object
oIsoMapDr = Isomap(n_neighbors = numNeighbors, n_components = lowDim, metric = metricType)
# Build the model
oIsoMapDr = oIsoMapDr.fit(mX)

* <font color='red'>(**?**)</font> Does this method support out of sample data? Look for `transform()` method.

In [ ]:
mZ = oIsoMapDr.transform(mX)

In [ ]:
# Plot the Low Dimensional Data (With the Faces)

# Compute Images which are far apart

lIdx = [0] #<! First image
lSet = list(range(1, numSamples)) #<! All sample but the first
for ii in range(numSamples - 1):
    mDi  = sp.spatial.distance.cdist(mZ[lIdx, :], mZ[lSet, :], metric = 'sqeuclidean') #<! Faster than the default 'euclidean'
    vMin = np.min(mDi, axis = 0) #<! i - Selected, j - Set
    idx  = np.argmax(vMin) #<! Image with the maximal minimum distance to the selected list
    lIdx.append(lSet[idx])
    lSet.remove(lSet[idx])

In [ ]:
# Plot the Embedding with Images

hF, hA = plt.subplots(figsize = (10, 8))

imgShift = 5
for ii in range(numImgScatter):
    idx = lIdx[ii]
    x0  = mZ[idx, 0] - imgShift
    x1  = mZ[idx, 0] + imgShift
    y0  = mZ[idx, 1] - imgShift
    y1  = mZ[idx, 1] + imgShift
    mI  = np.reshape(mX[idx, :], tImgSize)
    hA.imshow(mI, aspect = 'auto', cmap = 'gray', zorder = 2, extent = (x0, x1, y0, y1))

hA.scatter(mZ[:, 0], mZ[:, 1], s = 50, c = 'lime', edgecolor = 'k')
hA.set_xlabel('$z_1$')
hA.set_ylabel('$z_2$')

plt.show()

* <font color='blue'>(**!**)</font> Use Linear PCA to do the above and compare results.